In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow as tf
import os
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

/home/heng/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [4]:
# xdata2 = np.concatenate([xdata,xdata[:,::-1,:]],axis=2)
# ydata2 = np.concatenate([ydata,ydata[:,::-1,:]],axis=2)

In [37]:
# x = np.arange(28*28).reshape(28,28)

def snake(a):
    n,m =a.shape
    old_row = 0
    data = []
    for i in range(0,np.size(a),2):
        row = int(i/n)
        column1 = i%m
        column2 = (i+1)%m
#         print(row,column1,column2)
        if old_row!=row:
            a = np.flip(a,axis=1)
#         print(a[row,column1:column2+1])
        old_row = row
        d = a[row,column1:column2+1]
        data.append(d)
    return np.array(data)

def get_x(x):
    pad = np.zeros((28+2,28+2))
    pad[1:-1,1:-1] = x
    
    ax = snake(pad)
    ay = snake(pad.T)
    a = np.concatenate([ax,ay],axis=1)
    
    pad = np.rot90(pad)
    bx = snake(pad)
    by = snake(pad.T)
    b = np.concatenate([bx,by],axis=1)
    
    pad = np.rot90(pad)
    cx = snake(pad)
    cy = snake(pad.T)
    c = np.concatenate([cx,cy],axis=1)
    
    pad = np.rot90(pad)
    dx = snake(pad)
    dy = snake(pad.T)
    d = np.concatenate([dx,dy],axis=1)

    return [a,b,c,d]

In [49]:
xdata = np.array([get_x(x_train[i]) for i in range(0,2)])

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(np.rot90(np.rot90(np.rot90(x_train[1]))), interpolation='nearest')

In [ ]:
# generate data
np.random.seed(0)
latent_size = 5
n = 51
num_steps =5
n_inputs = 1

cycles = 25
stdev = .07
inputs = np.linspace(0,2*np.pi*cycles,n)
noise = np.random.randn(n)*stdev
true_data = (5*np.sin(inputs)+30*np.sin(5*inputs/cycles)+20*np.cos(20*inputs/cycles)+2*inputs)/(2*np.pi*cycles)
obs_data = true_data+noise
obs_data = (obs_data-obs_data.min()*1.01)/(obs_data.max()-obs_data.min()*.8)
ydata = pd.DataFrame(obs_data.reshape(-1,1)).shift(-1).dropna().values.reshape(1,-1,1)
xdata = obs_data[:-1].reshape(1,-1,1)
t = np.arange(xdata.size).reshape(xdata.shape)
len(xdata)==len(ydata)

# xdata = np.array(l).reshape(-1,100,1)
# ydata = np.roll(xdata,-1)

In [ ]:
self.input2 =  tf.concat([self.x2_list[t],self.H],axis=0) 

In [ ]:
self.input3 =  tf.concat([self.x3_list[t],self.H],axis=0) 

In [ ]:
self.input4 =  tf.concat([self.x4_list[t],self.H],axis=0) 

In [197]:
class GridLSTM(object):
    def __init__(self,units,grid_size,num_classes,max_steps,forget_bias=1):
        tf.set_random_seed(1)
        self.num_units = units
        self.all_loss = []
        self.clip_grad_norm = True
        self.max_norm_gradient = 1.0
        self.num_classes = num_classes
        self.grid_size = grid_size
        forget_bias = 1.0
        self.dtype = tf.float32
        
        self.x1 = tf.placeholder(dtype=self.dtype,shape=[None,None,self.num_classes])
        self.x2 = tf.placeholder(dtype=self.dtype,shape=[None,None,self.num_classes])
        self.x3 = tf.placeholder(dtype=self.dtype,shape=[None,None,self.num_classes])
        self.x4 = tf.placeholder(dtype=self.dtype,shape=[None,None,self.num_classes])
        
        self.y = tf.placeholder(dtype=self.dtype,shape=[None,None,self.num_classes])
        self.global_step = tf.Variable(0, name='global_step', trainable=False)
        self.lr = tf.placeholder(dtype=self.dtype,name='learning_rate')
        self.batch_size = tf.shape(self.x1)[0]
        self.max_steps = max_steps
        self.x1_list = [tf.squeeze(x, axis=[1])
                  for x in tf.split(
                    axis=1, num_or_size_splits=max_steps,
                    value=tf.transpose(self.x1), name="inputs_list")]
        self.x2_list = [tf.squeeze(x, axis=[1])
                  for x in tf.split(
                    axis=1, num_or_size_splits=max_steps,
                    value=tf.transpose(self.x2), name="inputs_list")]
        self.x3_list = [tf.squeeze(x, axis=[1])
                  for x in tf.split(
                    axis=1, num_or_size_splits=max_steps,
                    value=tf.transpose(self.x3), name="inputs_list")]
        self.x4_list = [tf.squeeze(x, axis=[1])
                  for x in tf.split(
                    axis=1, num_or_size_splits=max_steps,
                    value=tf.transpose(self.x4), name="inputs_list")]        
        self.hiddens = []
        self.cells = []
        self.activation = tf.nn.relu
        self.w_o = tf.get_variable('w_o',[self.grid_size*2,self.num_classes])
        self.b_o = tf.get_variable('b_o',[self.num_classes])
        
        #first layer
        self.Wu =tf.get_variable('Wu', [self.num_units*self.grid_size, self.grid_size*self.num_units*4+self.num_classes])
        self.Wf = tf.get_variable('Wf', [self.num_units*self.grid_size, self.grid_size*self.num_units*4+self.num_classes])
        self.Wo = tf.get_variable('Wo', [self.num_units*self.grid_size, self.grid_size*self.num_units*4+self.num_classes])
        self.Wc = tf.get_variable('Wc', [self.num_units*self.grid_size, self.grid_size*self.num_units*4+self.num_classes])
        
        self.bu=tf.get_variable('bu',[self.num_units*self.grid_size,self.num_units])
        self.bf=tf.get_variable('bf',[self.num_units*self.grid_size,self.num_units],initializer=tf.constant_initializer(forget_bias))
        self.bo=tf.get_variable('bo',[self.num_units*self.grid_size,self.num_units])
        self.bc=tf.get_variable('bc',[self.num_units*self.grid_size,self.num_units])

        #second layer
        self.Wu2 =tf.get_variable('Wu2', [self.num_units*self.grid_size, self.grid_size*self.num_units*4+self.num_classes])
        self.Wf2 = tf.get_variable('Wf2', [self.num_units*self.grid_size, self.grid_size*self.num_units*4+self.num_classes])
        self.Wo2 = tf.get_variable('Wo2', [self.num_units*self.grid_size, self.grid_size*self.num_units*4+self.num_classes])
        self.Wc2 = tf.get_variable('Wc2', [self.num_units*self.grid_size, self.grid_size*self.num_units*4+self.num_classes])
        
        self.bu2=tf.get_variable('bu2',[self.num_units*self.grid_size,self.num_units])
        self.bf2=tf.get_variable('bf2',[self.num_units*self.grid_size,self.num_units],initializer=tf.constant_initializer(forget_bias))
        self.bo2=tf.get_variable('bo2',[self.num_units*self.grid_size,self.num_units])
        self.bc2=tf.get_variable('bc2',[self.num_units*self.grid_size,self.num_units])
        
        #third layer
        self.Wu3 =tf.get_variable('Wu3', [self.num_units*self.grid_size, self.grid_size*self.num_units*4+self.num_classes])
        self.Wf3 = tf.get_variable('Wf3', [self.num_units*self.grid_size, self.grid_size*self.num_units*4+self.num_classes])
        self.Wo3 = tf.get_variable('Wo3', [self.num_units*self.grid_size, self.grid_size*self.num_units*4+self.num_classes])
        self.Wc3 = tf.get_variable('Wc3', [self.num_units*self.grid_size, self.grid_size*self.num_units*4+self.num_classes])
        
        self.bu3=tf.get_variable('bu3',[self.num_units*self.grid_size,self.num_units])
        self.bf3=tf.get_variable('bf3',[self.num_units*self.grid_size,self.num_units],initializer=tf.constant_initializer(forget_bias))
        self.bo3=tf.get_variable('bo3',[self.num_units*self.grid_size,self.num_units])
        self.bc3=tf.get_variable('bc3',[self.num_units*self.grid_size,self.num_units]) 

        #fourth layer
        self.Wu4 =tf.get_variable('Wu4', [self.num_units*self.grid_size, self.grid_size*self.num_units*4+self.num_classes])
        self.Wf4 = tf.get_variable('Wf4', [self.num_units*self.grid_size, self.grid_size*self.num_units*4+self.num_classes])
        self.Wo4 = tf.get_variable('Wo4', [self.num_units*self.grid_size, self.grid_size*self.num_units*4+self.num_classes])
        self.Wc4 = tf.get_variable('Wc4', [self.num_units*self.grid_size, self.grid_size*self.num_units*4+self.num_classes])
        
        self.bu4=tf.get_variable('bu4',[self.num_units*self.grid_size,self.num_units])
        self.bf4=tf.get_variable('bf4',[self.num_units*self.grid_size,self.num_units],initializer=tf.constant_initializer(forget_bias))
        self.bo4=tf.get_variable('bo4',[self.num_units*self.grid_size,self.num_units])
        self.bc4=tf.get_variable('bc4',[self.num_units*self.grid_size,self.num_units])
        
#         #second layer
#         self.Wu2 =tf.get_variable('Wu2', [self.num_units*self.grid_size, self.grid_size*self.num_units+self.grid_size*2])
#         self.Wf2 = tf.get_variable('Wf2', [self.num_units*self.grid_size, self.grid_size*self.num_units+self.grid_size*2])
#         self.Wo2 = tf.get_variable('Wo2', [self.num_units*self.grid_size, self.grid_size*self.num_units+self.grid_size*2])
#         self.Wc2 = tf.get_variable('Wc2', [self.num_units*self.grid_size, self.grid_size*self.num_units+self.grid_size*2])
        
#         self.bu2=tf.get_variable('bu2',[self.num_units*self.grid_size,self.num_units])
#         self.bf2=tf.get_variable('bf2',[self.num_units*self.grid_size,self.num_units],initializer=tf.constant_initializer(forget_bias))
#         self.bo2=tf.get_variable('bo2',[self.num_units*self.grid_size,self.num_units])
#         self.bc2=tf.get_variable('bc2',[self.num_units*self.grid_size,self.num_units])        
        

        
        self.H1 =  tf.zeros((grid_size,self.num_units,self.batch_size))
        self.M1 = tf.zeros((grid_size,self.num_units,self.batch_size))
        self.H2 =  tf.zeros((grid_size,self.num_units,self.batch_size))
        self.M2 = tf.zeros((grid_size,self.num_units,self.batch_size))
        self.H3 =  tf.zeros((grid_size,self.num_units,self.batch_size))
        self.M3 = tf.zeros((grid_size,self.num_units,self.batch_size))
        self.H4 =  tf.zeros((grid_size,self.num_units,self.batch_size))
        self.M4 = tf.zeros((grid_size,self.num_units,self.batch_size))    
        self.H = tf.concat([self.H1,self.H2,self.H3,self.H4],axis=0)
        self.M = tf.concat([self.M1,self.M2,self.M3,self.M4],axis=0)        
        with tf.variable_scope('cell') as scope:           
            for t in range(0,2):
                if t>0: scope.reuse_variables()
                #first layer
                self.H = tf.reshape(self.H,shape=[-1,self.num_units*self.batch_size])
                self.input1 =  tf.concat([self.x1_list[t],self.H],axis=0) 
#                 self.Gu = tf.sigmoid(tf.matmul(self.Wu,self.input1)+self.bu)
#                 self.Gf = tf.sigmoid(tf.matmul(self.Wf,self.input1)+self.bf)
#                 self.Go = tf.sigmoid(tf.matmul(self.Wo,self.input1)+self.bo)
#                 self.Gc = self.activation(tf.matmul(self.Wc,self.input1)+self.bc)

#                 self.M_new1 = tf.multiply(self.Gu,self.Gc)+tf.multiply(self.Gf,self.M1)
#                 self.H_new1 = self.activation(tf.multiply(self.Go,self.M_new1))
#                 self.M1 = self.M_new1
#                 self.H1 = self.H_new1
                
#                 self.input2 =  tf.concat([self.x2_list[t],self.H],axis=0) 
#                 self.Gu2 = tf.sigmoid(tf.matmul(self.Wu2,self.input2)+self.bu2)
#                 self.Gf2 = tf.sigmoid(tf.matmul(self.Wf2,self.input2)+self.bf2)
#                 self.Go2 = tf.sigmoid(tf.matmul(self.Wo2,self.input2)+self.bo2)
#                 self.Gc2 = self.activation(tf.matmul(self.Wc2,self.input2)+self.bc2)

#                 self.M_new2 = tf.multiply(self.Gu2,self.Gc2)+tf.multiply(self.Gf2,self.M2)
#                 self.H_new2 = self.activation(tf.multiply(self.Go2,self.M_new2))
#                 self.M2 = self.M_new2
#                 self.H2 = self.H_new2                
                
#                 self.input3 =  tf.concat([self.x3_list[t],self.H],axis=0) 
#                 self.Gu3 = tf.sigmoid(tf.matmul(self.Wu3,self.input3)+self.bu3)
#                 self.Gf3 = tf.sigmoid(tf.matmul(self.Wf3,self.input3)+self.bf3)
#                 self.Go3 = tf.sigmoid(tf.matmul(self.Wo3,self.input3)+self.bo3)
#                 self.Gc3 = self.activation(tf.matmul(self.Wc3,self.input3)+self.bc3)

#                 self.M_new3 = tf.multiply(self.Gu3,self.Gc3)+tf.multiply(self.Gf3,self.M3)
#                 self.H_new3 = self.activation(tf.multiply(self.Go3,self.M_new3))
#                 self.M3 = self.M_new3
#                 self.H3 = self.H_new3
                
#                 self.input4 =  tf.concat([self.x4_list[t],self.H],axis=0) 
#                 self.Gu4 = tf.sigmoid(tf.matmul(self.Wu4,self.input4)+self.bu4)
#                 self.Gf4 = tf.sigmoid(tf.matmul(self.Wf4,self.input4)+self.bf4)
#                 self.Go4 = tf.sigmoid(tf.matmul(self.Wo4,self.input4)+self.bo4)
#                 self.Gc4 = self.activation(tf.matmul(self.Wc4,self.input4)+self.bc4)

#                 self.M_new4 = tf.multiply(self.Gu4,self.Gc4)+tf.multiply(self.Gf4,self.M4)
#                 self.H_new4 = self.activation(tf.multiply(self.Go4,self.M_new4))
#                 self.M4 = self.M_new4
#                 self.H4 = self.H_new4             

#                 self.H = tf.concat([self.H1,self.H2,self.H3,self.H4],axis=0)
#                 self.M = tf.concat([self.M1,self.M2,self.M3,self.M4],axis=0)
                
                
                
#                 #second layer
#                 self.input2 = tf.concat([tf.concat([self.H,self.M],axis=0),self.H2],axis=0)
#                 self.Gu2 = tf.sigmoid(tf.matmul(self.Wu2,self.input2)+self.bu2)
#                 self.Gf2 = tf.sigmoid(tf.matmul(self.Wf2,self.input2)+self.bf2)
#                 self.Go2 = tf.sigmoid(tf.matmul(self.Wo2,self.input2)+self.bo2)
#                 self.Gc2 = self.activation(tf.matmul(self.Wc2,self.input2)+self.bc2)

#                 self.M_new2 = tf.multiply(self.Gu2,self.Gc2)+tf.multiply(self.Gf2,self.M2)
#                 self.H_new2 = self.activation(tf.multiply(self.Go2,self.M_new2))
#                 self.M2 = self.M_new2
#                 self.H2 = self.H_new2             

#                 self.hiddens.append(self.H)
#                 self.cells.append(self.M)

#         self.hiddens = tf.concat(self.hiddens,axis=1)
#         self.cells = tf.concat(self.cells,axis=1)
#         self.cell_hidden = tf.concat([self.cells,self.hiddens],axis=0)
        
#         self.outputs = self.activation(tf.matmul(self.cell_hidden,self.w_o)+self.b_o)
#         self.outputs = tf.reshape(self.outputs,shape=tf.shape(self.x))
        

#         self.loss = tf.losses.mean_squared_error(labels=self.y,predictions=self.outputs)
#         self.optimizer = tf.train.AdamOptimizer(learning_rate=self.lr)

#         self.grads_vars =self.optimizer.compute_gradients(self.loss)
    
#         self.train_op = self.optimizer.apply_gradients(self.grads_vars,
#                                                            global_step=self.global_step)

    def train_step(self,sess,rate):
        _, train_step, train_loss = sess.run([self.train_op,
                        self.global_step,
                        self.loss],
                        {self.x:xdata2,
                          self.y:ydata2,
                           self.lr:rate})
        self.all_loss.append(train_loss)
        return train_step, train_loss
        
    def get_output(self,sess):
        return sess.run([self.outputs,
                                    self.hiddens,
                                    self.cells],{self.x:xdata2,self.y:ydata2})
    

In [198]:
tf.reset_default_graph()
tf.set_random_seed(1)
units =1
grid_size = 2
num_classes = 4
# max_steps = np.shape(xdata2)[1]
max_steps = np.shape(xdata[0])[1]
m = GridLSTM(units,grid_size,num_classes,max_steps)

######
output_dir = "./output/"

#######
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.25)
sess = tf.Session(config=tf.ConfigProto(
        gpu_options=gpu_options,
        log_device_placement=False
    ))
init = tf.global_variables_initializer()
sess.run(init)

In [199]:
feed_dict = {m.x1:xdata[:,0,:,:],m.x2:xdata[:,1,:,:],m.x3:xdata[:,2,:,:],m.x4:xdata[:,3,:,:]}

In [203]:
sess.run( tf.sigmoid(tf.matmul(m.Wu,m.input1))+m.bu,,feed_dict).shape

(2, 2)

In [190]:
sess.run(m.H,feed_dict).shape

(8, 2)

In [54]:
feed_dict = {m.x1:xdata[:,0,:,:],m.x2:xdata[:,1,:,:],m.x3:xdata[:,2,:,:],m.x4:xdata[:,3,:,:]}
a = sess.run(m.hiddens,feed_dict)
b = sess.run(m.cells,feed_dict)
c = sess.run(m.cell_hidden,feed_dict)


InvalidArgumentError: ConcatOp : Dimensions of inputs should match: shape[0] = [4,2] vs. shape[1] = [8,1]
	 [[{{node cell/concat_3}} = ConcatV2[N=2, T=DT_FLOAT, Tidx=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](cell/transpose_3, concat, cell/concat_2694/axis)]]

Caused by op 'cell/concat_3', defined at:
  File "/home/heng/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/heng/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/heng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/heng/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/heng/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/home/heng/anaconda3/lib/python3.6/site-packages/tornad

In [30]:
c.flatten()

array([0.        , 0.        , 0.        , ..., 0.0014872 , 0.00148466,
       0.00148218], dtype=float32)

In [ ]:
a = np.concatenate(a,axis=1)
b = np.concatenate(b,axis=1)

In [ ]:
np.concatenate([a,b],axis=0).shape

In [ ]:
tf.reset_default_graph()
tf.set_random_seed(1)
units =1
grid_size = 2
num_classes = 1
# max_steps = np.shape(xdata2)[1]
max_steps = np.shape(xdata)[1]
m = GridLSTM(units,grid_size,num_classes,max_steps)

######
output_dir = "./output/"

#######
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.25)
sess = tf.Session(config=tf.ConfigProto(
        gpu_options=gpu_options,
        log_device_placement=False
    ))
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
tf.reset_default_graph()
tf.set_random_seed(1)
units =1
grid_size = 2
num_classes = 2
max_steps = np.shape(xdata2)[1]
m = GridLSTM(units,grid_size,num_classes,max_steps)

######
output_dir = "./output/"

#######
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.25)
sess = tf.Session(config=tf.ConfigProto(
        gpu_options=gpu_options,
        log_device_placement=False
    ))
init = tf.global_variables_initializer()
sess.run(init)

# summary_out_dir = os.path.join(output_dir, "summaries")
# summary_writer  = tf.summary.FileWriter(summary_out_dir, sess.graph)

rate = .01
epochs=5000
for e in range(epochs):
    train_step, train_loss = m.train_step(sess,rate)
#     summary_writer.add_summary(train_summary, train_step)
    
    if e%50==0:
        print(train_loss)
        output_list = m.get_output(sess)
# output_list = m.get_output(sess)
# sess.close()
pd.DataFrame([output_list[0].flatten(),ydata.flatten()]).T.plot()

In [ ]:
output_list[0]

In [ ]:
pd.DataFrame([output_list[0].flatten(),ydata.flatten()]).T.plot()